In [2]:
# Imports
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys
from syspath import current_path
sys.path.append('../../')

import numpy as np
import pandas as pd 
import os
import pickle
import datetime
import time
import scipy.sparse as sp
import matplotlib.pyplot as plt
%matplotlib inline

from gensim.models import Word2Vec

In [2]:
# raw Last.fm data available at http://ocelma.net/MusicRecommendationDataset/
# PATH = '../../data/lastfm/raw/userid-timestamp-artid-artname-traid-traname.tsv'
# cols = ['userId', 'timestamp', 'artistId', 'artist-name','songId', 'song-name']                   

df = pd.read_csv(PATH, sep="\t", names=cols, error_bad_lines=False)

In [3]:
large_song_df = large_song_df[large_song_df.songId.notnull()]
large_song_df = large_song_df[large_song_df.artistId.notnull()]

# convert proper date/time
large_song_df['timestamp'] = pd.to_datetime(large_song_df.timestamp)
large_song_df["date"] = [d.date() for d in large_song_df["timestamp"]]

users = list(large_song_df.userId.unique())
user_dict = dict(zip(users,[i for i in range(len(users))]))
large_song_df['userId'] = large_song_df['userId'].apply(lambda x: user_dict[x])

items = list(large_song_df.artistId.unique())
item_dict = dict(zip(items,[i for i in range(len(items))]))
large_song_df['artistId'] = large_song_df['artistId'].apply(lambda x: item_dict[x])

n = 3

# df = bet_df.groupby('league_id')['placed_date'].count()
artists = large_song_df.groupby('artistId')['userId'].nunique()
prev_len = len(artists)
artists = artists[artists>n].index.tolist()
large_song_df = large_song_df.query('artistId in @artists')
print('Removed {} artist ids to {}'.format(prev_len-len(artists),len(artists)))

n = 5

# df = bet_df.groupby('league_id')['placed_date'].count()
users = large_song_df.userId.value_counts()
prev_len = len(users)
users = users[users>n].index.tolist()
large_song_df = large_song_df.query('userId in @users')
print('Removed {} user ids to {}'.format(prev_len-len(users),len(users)))

Removed 57130 artist ids to 26775
Removed 3 user ids to 989


In [4]:
dates = sorted(list(large_song_df['date'].unique()))
len(dates)

train = 24
val = 7
test = 8
total = train+val+test
n_days = len(dates)
train_days = int(n_days*train/total)
val_days = int(n_days*val/total)
test_days = n_days - (train_days+val_days)
print(train_days, val_days, test_days, n_days)

977 285 327 1589


In [5]:
large_song_df

,userId,timestamp,artistId,artist-name,songId,song-name,date
10,0,2009-05-04 13:06:09+00:00,0,坂本龍一,f7c1f8f8-b935-45ed-8fc8-7def69d92a10,The Last Emperor (Theme),2009-05-04
12,0,2009-05-04 12:55:34+00:00,0,坂本龍一,475d4e50-cebb-4cd0-8cd4-c3df97987962,Tibetan Dance (Version),2009-05-04
14,0,2009-05-03 15:48:25+00:00,1,Underworld,dc394163-2b78-4b56-94e4-658597a29ef8,"Boy, Boy, Boy (Switch Remix)",2009-05-03
15,0,2009-05-03 15:37:56+00:00,1,Underworld,340d9a0b-9a43-4098-b116-9f79811bd508,Crocodile (Innervisions Orchestra Mix),2009-05-03
16,0,2009-05-03 15:14:53+00:00,2,Ennio Morricone,0b04407b-f517-4e00-9e6a-494795efc73e,Ninna Nanna In Blu (Raw Deal Remix),2009-05-03
...,...,...,...,...,...,...,...
19098857,991,2008-01-27 22:02:35+00:00,1161,Wilco,a490cabc-1e5c-4807-86c7-740c31a50009,Please Be Patient With Me,2008-01-27
19098858,991,2008-01-27 21:56:52+00:00,1161,Wilco,3e92e447-9e1f-440d-bc00-6734469880c5,Shake It Off,2008-01-27
19098859,991,2008-01-27 21:52:36+00:00,1161,Wilco,93d044e6-1bbb-46a6-ac8e-283382a89e6f,Side With The Seeds,2008-01-27
19098860,991,2008-01-27 21:49:12+00:00,1161,Wilco,5ac4386f-6146-4389-a762-4b43f362d2c8,Sky Blue Sky,2008-01-27


In [6]:
df = large_song_df[['userId','artistId','timestamp','date']]
df = df[df['date'] <= dates[-(test_days+val_days)]]
df = df.sort_values(by='timestamp')
df['artistId'].nunique()

24937

In [7]:
df["time"] = [d.time() for d in df["timestamp"]]
df

,userId,artistId,timestamp,date,time
7558604,387,1126,2005-02-14 00:00:07+00:00,2005-02-14,00:00:07
16881016,863,43835,2005-02-14 00:00:38+00:00,2005-02-14,00:00:38
13492631,701,3119,2005-02-14 00:01:44+00:00,2005-02-14,00:01:44
5577102,283,775,2005-02-14 00:02:10+00:00,2005-02-14,00:02:10
2859034,141,5317,2005-02-14 00:02:40+00:00,2005-02-14,00:02:40
...,...,...,...,...,...
12248254,654,1641,2007-10-19 23:59:33+00:00,2007-10-19,23:59:33
11502664,605,5352,2007-10-19 23:59:39+00:00,2007-10-19,23:59:39
4354102,223,2397,2007-10-19 23:59:42+00:00,2007-10-19,23:59:42
7321559,373,3251,2007-10-19 23:59:44+00:00,2007-10-19,23:59:44


In [8]:
new_df = df.copy().groupby(['userId','date']).agg(artist_per_day=('artistId', lambda x: list(set(x))))
new_df

artist_per_day
userId date                                                         
0      2006-08-13  [481, 482, 427, 11, 77, 462, 239, 496, 497, 49...
       2006-08-15  [131, 195, 488, 490, 427, 492, 493, 494, 495, ...
       2006-08-16      [481, 482, 195, 484, 485, 486, 483, 427, 244]
       2006-08-18                                              [464]
       2006-08-20                                              [464]
...                                                              ...
989    2007-10-15                                         [1162, 61]
       2007-10-16                                    [649, 61, 3255]
       2007-10-17                                   [1930, 2389, 61]
       2007-10-18                                       [3016, 1162]
       2007-10-19                                  [857, 1162, 6814]

[185911 rows x 1 columns]

In [9]:
new_df['artist_per_day'] = new_df['artist_per_day'].apply(lambda x: [str(i) for i in x])
new_df['artist_per_day'] = new_df['artist_per_day'].apply(lambda x: ",".join(x))
new_df

artist_per_day
userId date                                                         
0      2006-08-13  481,482,427,11,77,462,239,496,497,498,499,244,...
       2006-08-15  131,195,488,490,427,492,493,494,495,239,491,82...
       2006-08-16                481,482,195,484,485,486,483,427,244
       2006-08-18                                                464
       2006-08-20                                                464
...                                                              ...
989    2007-10-15                                            1162,61
       2007-10-16                                        649,61,3255
       2007-10-17                                       1930,2389,61
       2007-10-18                                          3016,1162
       2007-10-19                                      857,1162,6814

[185911 rows x 1 columns]

In [10]:
new_df.to_pickle('../../data/lastfm/processed/item2vec_train_ex_val.pkl')
new_df = pd.read_pickle('../../data/lastfm/processed/item2vec_train_full.pkl')

In [15]:
with open('../../data/lastfm/processed/test_users_map.txt', 'rb') as f:
    # read the data as binary data stream
    items = pickle.load(f)

In [19]:
class ItemSequences:
    '''
    ItemSequences
    '''
    def __init__(self, df_column):
        df_column = df_column.copy()
        self.sequences = df_column.values.tolist()

    def __iter__(self):
        for sequence in self.sequences:
            yield sequence.split(',')

In [20]:
sentences = ItemSequences(new_df.artist_per_day)
model = Word2Vec(sentences, window=5, min_count=5, sg=1, size=64)

In [21]:
valid_items = list(map(int,model.wv.index2word))
valid_dict = {j:i for i,j in enumerate(valid_items)}
inter = sorted(list(set(valid_items)&set(items)))

inter_embed = [valid_dict[i] for i in inter]
valid_embeds = model.wv.vectors[inter_embed]
valid_embeds.shape

In [1]:
new_full_embeds = np.zeros(shape=(len(items), 64))

for i in range(len(items)):
    if i in inter:
        new_full_embeds[i,:] = valid_embeds[inter.index(i),:]

In [27]:
np.save('../../gcmc/src/data/lastfm/side_info/info.npy', new_full_embeds)

In [28]:
items_in_train_dict = {j:i for i,j in enumerate(items)}
valid_inter_mapped = [items_in_train_dict[i] for i in inter]

In [30]:
with open('../../gcmc/src/data/lastfm/side_info/valid_items.txt', "wb") as fp:
    pickle.dump(valid_inter_mapped, fp)